In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
data = pd.read_csv("../../data/processed/pagos_por_unidad.csv",
                  dtype = {
                                "unidad_id": "object",
                            },
                  parse_dates = ["expensa_mes_pago"])

In [3]:
pagos_test = data.loc[data.expensa_mes_pago >= "2018-09-01"]
pagos_cv = data.loc[(data.expensa_mes_pago < "2018-09-01") & (data.expensa_mes_pago >= "2018-06-01")]
pagos_train = data.loc[data.expensa_mes_pago < "2018-06-01"]

pagos_train_cv = data.loc[data.expensa_mes_pago < "2018-09-01"]

X_train = pagos_train.drop(columns=["unidad_id","expensa_mes_pago", "target"])
Y_train = pagos_train.target
X_cv = pagos_cv.drop(columns=["unidad_id","expensa_mes_pago", "target"])
Y_cv = pagos_cv.target
X_test = pagos_test.drop(columns=["unidad_id","expensa_mes_pago", "target"])
Y_test = pagos_test.target

print(X_train.shape, Y_train.shape)
print(X_cv.shape, Y_cv.shape)
print(X_test.shape, Y_test.shape)

(108621, 6) (108621,)
(23226, 6) (23226,)
(37108, 6) (37108,)


In [4]:
words_dict = {
    "Impago":0,
    "Internet":1,
    "EntePago":2,
    "Efec-Cheque":3,
    "Otro":4,
    "NS/NC":5
}

def transform_values(X_i):
    for column in X_i.columns:
        X_i[column] = X_i[column].map(lambda x: words_dict[x])
    return X_i

In [5]:
X_train = transform_values(X_train)
X_cv = transform_values(X_cv)
X_test = transform_values(X_test)

In [6]:
print(X_train.shape, Y_train.shape)
print(X_cv.shape, Y_cv.shape)
print(X_test.shape, Y_test.shape)

(108621, 6) (108621,)
(23226, 6) (23226,)
(37108, 6) (37108,)


In [7]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def get_train_test_meassures(model, X_train, Y_train, X_test, Y_test):
    train_preds = model.predict(X_train)
    print("accuracy:", accuracy_score(Y_train, train_preds))
    print("precision:", precision_score(Y_train, train_preds))
    print("recall:", recall_score(Y_train, train_preds))
    print("f1:", f1_score(Y_train, train_preds))
    print("--------------------------------------------------")
    test_preds = model.predict(X_test)
    print("accuracy:", accuracy_score(Y_test, test_preds))
    print("precision:", precision_score(Y_test, test_preds))
    print("recall:", recall_score(Y_test, test_preds))
    print("f1:", f1_score(Y_test, test_preds))

In [8]:
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.models import Sequential
from keras.layers import LSTM, Embedding

def create_model(layer1=10, layer2=10, layer3=10, dropout=0.1):
    model = Sequential()
    model.add(Embedding(input_dim=6, output_dim=6, input_length=6))
    model.add(Flatten())
    model.add(Dense(layer1, activation="relu"))
    model.add(Dropout(dropout))
    model.add(Dense(layer2, activation="relu"))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["acc",f1])
    return model

In [9]:
data = data.sort_values("expensa_mes_pago").reset_index().drop(columns="index")

In [10]:
min_cv_id = data.loc[data.expensa_mes_pago == "2018-06-01", "expensa_mes_pago"].idxmin()
min_test_id = data.loc[data.expensa_mes_pago == "2018-09-01", "expensa_mes_pago"].idxmin()

X_train_cv = pagos_train_cv.drop(columns=["unidad_id","expensa_mes_pago", "target"])
Y_train_cv = pagos_train_cv.target
X_train_cv = transform_values(X_train_cv)
print(X_train_cv.shape, Y_train_cv.shape)

(131847, 6) (131847,)


In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
test = KerasClassifier(build_fn=create_model, verbose=5, layer1=10, layer2=10, layer3=10, dropout=0.1, epochs=3, batch_size = 128)

In [12]:
test.fit(X_train_cv, Y_train_cv)

Epoch 1/3
Epoch 2/3
Epoch 3/3


In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "layer1":[10,20]
}

neural_network = GridSearchCV(
    estimator = KerasClassifier(build_fn=create_model, verbose=5),
    param_grid = param_grid,
    scoring = 'f1',
    n_jobs = 1,
    cv = [(list(range(0,min_cv_id)), list(range(min_cv_id,min_test_id)))],
    return_train_score = True,
    error_score = 0,
    verbose = 5
)


In [18]:
neural_network.fit(X_train_cv.values, Y_train_cv.values)

Fitting 1 folds for each of 2 candidates, totalling 2 fits
[CV] layer1=10 .......................................................
Epoch 1/1
[CV] .............. layer1=10, score=0.7479874988161758, total=   6.2s
[CV] layer1=20 .......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.5s remaining:    0.0s


Epoch 1/1
[CV] .............. layer1=20, score=0.7412767644726407, total=   6.4s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.4s finished


Epoch 1/1


GridSearchCV(cv=[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, ...4, 131835, 131836, 131837, 131838, 131839, 131840, 131841, 131842, 131843, 131844, 131845, 131846])],
       error_score=0,
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x00000266146137B8>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'layer1': [10, 20]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='f1', verbose=5)

In [20]:
neural_network.best_score_

0.7479874988161758

In [24]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from keras.wrappers.scikit_learn import KerasClassifier

search_spaces = {
    "layer1": Integer(1,100),
    "layer2": Integer(1,100),
    "layer3": Integer(1,100),
    "dropout": Real(0,1),
    "epochs": Integer(5,50),
    "batch_size": Integer(50, 10000)
}

neural_network = BayesSearchCV(
    estimator = KerasClassifier(build_fn=create_model, verbose=5),
    search_spaces = search_spaces,
    n_iter = 30,
    scoring = 'f1',
    n_jobs = 1,
    cv = [(list(range(0,min_cv_id)), list(range(min_cv_id,min_test_id)))],
    return_train_score = True,
    error_score = 0,
    random_state = 42,
    verbose = 5
)

In [25]:
neural_network.fit(X_train_cv.values, Y_train_cv.values)

Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV] batch_size=4131, dropout=0.7277257431773251, epochs=47, layer1=32, layer2=67, layer3=42 
Epoch 1/47
Epoch 2/47
Epoch 3/47
Epoch 4/47
Epoch 5/47
Epoch 6/47
Epoch 7/47
Epoch 8/47
Epoch 9/47
Epoch 10/47
Epoch 11/47
Epoch 12/47
Epoch 13/47
Epoch 14/47
Epoch 15/47
Epoch 16/47
Epoch 17/47
Epoch 18/47
Epoch 19/47
Epoch 20/47
Epoch 21/47
Epoch 22/47
Epoch 23/47
Epoch 24/47
Epoch 25/47
Epoch 26/47
Epoch 27/47
Epoch 28/47
Epoch 29/47
Epoch 30/47
Epoch 31/47
Epoch 32/47
Epoch 33/47
Epoch 34/47
Epoch 35/47
Epoch 36/47
Epoch 37/47
Epoch 38/47
Epoch 39/47
Epoch 40/47
Epoch 41/47
Epoch 42/47
Epoch 43/47
Epoch 44/47
Epoch 45/47
Epoch 46/47
Epoch 47/47
[CV]  batch_size=4131, dropout=0.7277257431773251, epochs=47, layer1=32, layer2=67, layer3=42, score=0.747821034282394, total=  15.7s
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV] batch_size=8382, dropout=0.8833152773808622, epochs=19, layer1=95, layer2=87, layer3=7 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.8s finished


Epoch 1/19
Epoch 2/19
Epoch 3/19
Epoch 4/19
Epoch 5/19
Epoch 6/19
Epoch 7/19
Epoch 8/19
Epoch 9/19
Epoch 10/19
Epoch 11/19
Epoch 12/19
Epoch 13/19
Epoch 14/19
Epoch 15/19
Epoch 16/19
Epoch 17/19
Epoch 18/19
Epoch 19/19
[CV]  batch_size=8382, dropout=0.8833152773808622, epochs=19, layer1=95, layer2=87, layer3=7, score=0.7461493468512381, total=   9.1s
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV] batch_size=4476, dropout=0.9187225216693771, epochs=10, layer1=44, layer2=20, layer3=46 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.2s finished


Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
[CV]  batch_size=4476, dropout=0.9187225216693771, epochs=10, layer1=44, layer2=20, layer3=46, score=0.74293059125964, total=   3.9s
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV] batch_size=8133, dropout=0.1718715613965607, epochs=32, layer1=80, layer2=53, layer3=10 


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s finished


Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV] batch_size=5630, dropout=0.6622499588708453, epochs=10, layer1=55, layer2=29, layer3=77 
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
[CV]  batch_size=5630, dropout=0.6622499588708453, epochs=10, layer1=55, layer2=29, layer3=77, score=0.7473330129745316, total=   5.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.6s finished


Epoch 1/29
Epoch 2/29
Epoch 3/29
Epoch 4/29
Epoch 5/29
Epoch 6/29
Epoch 7/29
Epoch 8/29
Epoch 9/29
Epoch 10/29
Epoch 11/29
Epoch 12/29
Epoch 13/29
Epoch 14/29
Epoch 15/29
Epoch 16/29
Epoch 17/29
Epoch 18/29
Epoch 19/29
Epoch 20/29
Epoch 21/29
Epoch 22/29
Epoch 23/29
Epoch 24/29
Epoch 25/29
Epoch 26/29
Epoch 27/29
Epoch 28/29
Epoch 29/29


BayesSearchCV(cv=[([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, ...4, 131835, 131836, 131837, 131838, 131839, 131840, 131841, 131842, 131843, 131844, 131845, 131846])],
       error_score=0,
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002662FF046A0>,
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=42,
       refit=True, return_train_score=True, scoring='f1',
       search_spaces={'layer1': Integer(low=1, high=100), 'layer2': Integer(low=1, high=100), 'layer3': Integer(low=1, high=100), 'dropout': Real(low=0, high=1, prior='uniform', transform='identity'), 'epochs': Integer(low=5, high=50), 'batch_size': Integer(low=50, hig

In [27]:
neural_network.best_params_

{'batch_size': 5924,
 'dropout': 0.629900496028121,
 'epochs': 29,
 'layer1': 10,
 'layer2': 58,
 'layer3': 34}

In [37]:
get_train_test_meassures(neural_network, X_train_cv, Y_train_cv, X_test, Y_test)

accuracy: 0.8797924867460011
precision: 0.7943632875903792
recall: 0.7519135147214928
f1: 0.7725557166023277
--------------------------------------------------
accuracy: 0.8807534763393339
precision: 0.7660174269605331
recall: 0.6969802961408418
f1: 0.729869971308223


In [51]:
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Conv1D, MaxPool1D

model = Sequential()
model.add(Embedding(input_dim=6, output_dim=6, input_length=6))
model.add(Conv1D(filters=10, kernel_size=3))
model.add(MaxPool1D(2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["acc",f1])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_56 (Embedding)     (None, 6, 6)              36        
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 4, 10)             190       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 10)             0         
_________________________________________________________________
flatten_40 (Flatten)         (None, 20)                0         
_________________________________________________________________
dense_117 (Dense)            (None, 1)                 21        
Total params: 247
Trainable params: 247
Non-trainable params: 0
_________________________________________________________________


In [52]:
model.fit(X_train, Y_train, batch_size=128, validation_data=(X_cv, Y_cv), epochs=10)

Train on 108621 samples, validate on 23226 samples
Epoch 1/10
108621/108621 [==============================] - 5s 50us/step - loss: 0.3412 - acc: 0.8718 - f1: 0.7510 - val_loss: 0.2978 - val_acc: 0.8856 - val_f1: 0.6988
Epoch 2/10
108621/108621 [==============================] - 3s 29us/step - loss: 0.3118 - acc: 0.8788 - f1: 0.7690 - val_loss: 0.2967 - val_acc: 0.8876 - val_f1: 0.7008
Epoch 3/10
108621/108621 [==============================] - 3s 29us/step - loss: 0.3107 - acc: 0.8788 - f1: 0.7695 - val_loss: 0.2977 - val_acc: 0.8862 - val_f1: 0.7007
Epoch 4/10
108621/108621 [==============================] - 3s 29us/step - loss: 0.3104 - acc: 0.8792 - f1: 0.7693 - val_loss: 0.2976 - val_acc: 0.8873 - val_f1: 0.7007
Epoch 5/10
108621/108621 [==============================] - 3s 29us/step - loss: 0.3103 - acc: 0.8789 - f1: 0.7685 - val_loss: 0.2972 - val_acc: 0.8875 - val_f1: 0.6995
Epoch 6/10
108621/108621 [==============================] - 3s 29us/step - loss: 0.3101 - acc: 0.8792 - 

In [56]:
def get_train_test_meassures2(model, X_train, Y_train, X_test, Y_test):
    train_preds = model.predict_classes(X_train)
    print("accuracy:", accuracy_score(Y_train, train_preds))
    print("precision:", precision_score(Y_train, train_preds))
    print("recall:", recall_score(Y_train, train_preds))
    print("f1:", f1_score(Y_train, train_preds))
    print("--------------------------------------------------")
    test_preds = model.predict_classes(X_test)
    print("accuracy:", accuracy_score(Y_test, test_preds))
    print("precision:", precision_score(Y_test, test_preds))
    print("recall:", recall_score(Y_test, test_preds))
    print("f1:", f1_score(Y_test, test_preds))

In [57]:
get_train_test_meassures2(model, X_train_cv, Y_train_cv, X_test, Y_test)

accuracy: 0.8804523424878837
precision: 0.8159455027122493
recall: 0.7227219397731717
f1: 0.7665096435872367
--------------------------------------------------
accuracy: 0.8809421149078366
precision: 0.7867089480215084
recall: 0.6652675760755509
f1: 0.7209096651926722
